In [1]:
import numpy as np
import pandas as pd
import datetime
import json
import requests

import pprint as pp

In [2]:
data_directory = '../database/populate_and_update/mongodb/output'
granularity = 'M15' # This data contains currency pair closing prices in 15-minute intervals
price_types = 'BAM'
config_file = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forex/data/DEVELOPMENT.json'

In [3]:
df_previous = pd.read_parquet(data_directory + '/' + granularity + '.parquet').sort_values(by = ['instrument', 'time'])
df_previous = df_previous[df_previous['complete']].copy()

df = df_previous[['instrument', 'time']].copy()
df.head()

,instrument,time
2553676,AUD_USD,1020717900
2553677,AUD_USD,1020804300
2553678,AUD_USD,1020890700
2553679,AUD_USD,1020977100
2553680,AUD_USD,1021063500


In [4]:
dict_max_time_we_already_have_per_instrument = {}
for item in df.groupby('instrument')['time'].agg('max').reset_index().to_dict(orient = 'records'):
    dict_max_time_we_already_have_per_instrument[item['instrument']] = item['time']

pp.pprint(dict_max_time_we_already_have_per_instrument)

{'AUD_USD': 1744385400,
 'EUR_USD': 1744385400,
 'GBP_USD': 1744385400,
 'NZD_USD': 1744385400,
 'USD_CAD': 1744385400,
 'USD_CHF': 1744385400,
 'USD_JPY': 1744385400}


In [5]:
now = int(np.floor(datetime.datetime.now().timestamp()))
now

1744974456

In [6]:
#
# load config
#
with open(config_file) as f:
    config = json.load(f)

In [7]:
#
# define a function to produce the headers from the given config file
#
def get_oanda_headers(config):
    headers = {
        'Content-Type' : 'application/json',
        'Authorization' : 'Bearer ' + config['token'],
        'Accept-Datetime-Format' : config['oanda_date_time_format'],
    }
    return headers

In [8]:
#
# send a request to Oanda for historical candlestick values
#
def get_instrument_candlesticks(
    instrument,
    price_types,
    granularity,
    start_date,
    end_date,
    config,
    error_retry_interval = 3,
):
    headers = get_oanda_headers(config)
    url = config['server'] + '/v3/instruments/' + instrument + '/candles?price=' + price_types + '&granularity=' + granularity + '&from=' + str(start_date) + '&to=' + str(end_date)

    worked = False
    while not worked:
        try:
            r = requests.get(url, headers=headers)
            worked = True
        except:
            time.sleep(error_retry_interval)
    
    rj = r.json()
    return rj

In [10]:
for instrument in ['AUD_USD']:

    start_time = dict_max_time_we_already_have_per_instrument[instrument]
    end_time = now

    results = get_instrument_candlesticks(instrument, price_types, granularity, start_time, end_time, config)
    candlesticks_temp = results['candles']
    
    candlesticks = []
    for c in candlesticks_temp:
        time_as_int = int(round(float(c['time']), 0))
        if c['complete']:
            if time_as_int != start_time:
                c['time'] = time_as_int
                candlesticks.append(c)

    print(instrument)
    pp.pprint(candlesticks[0:2])
    print()

AUD_USD
[{'ask': {'c': '0.62439', 'h': '0.62488', 'l': '0.62386', 'o': '0.62445'},
  'bid': {'c': '0.62424', 'h': '0.62475', 'l': '0.62373', 'o': '0.62431'},
  'complete': True,
  'mid': {'c': '0.62432', 'h': '0.62481', 'l': '0.62380', 'o': '0.62438'},
  'time': 1744386300,
  'volume': 3929},
 {'ask': {'c': '0.62435', 'h': '0.62466', 'l': '0.62362', 'o': '0.62441'},
  'bid': {'c': '0.62422', 'h': '0.62451', 'l': '0.62347', 'o': '0.62428'},
  'complete': True,
  'mid': {'c': '0.62428', 'h': '0.62458', 'l': '0.62355', 'o': '0.62434'},
  'time': 1744387200,
  'volume': 3394}]

